In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 5.0 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install datasets

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

In [ ]:
from transformers import AdamWeightDecay

In [ ]:
import numpy as np

In [ ]:
import evaluate

In [ ]:
from datasets import load_dataset

In [ ]:
from transformers import AutoTokenizer

In [ ]:
from transformers import DataCollatorForSeq2Seq

In [ ]:
data = load_dataset("arinzeo/indo-eng")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

indo_eng.csv:   0%|          | 0.00/2.55M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/36237 [00:00<?, ? examples/s]

In [ ]:
train_dataset = data["train"]

In [ ]:
train_dataset[:5]

{'Unnamed: 0': [0, 1, 2, 3, 4],
 'indo': ['Hai.', 'Lari!', 'Lari!', 'Siapa?', 'Wow!'],
 'eng': ['Hi.', 'Run!', 'Run.', 'Who?', 'Wow!']}

In [ ]:
len(train_dataset)

36237

In [ ]:
data_load = train_dataset.train_test_split(test_size=0.2)

In [ ]:
checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
source_lang = "eng"
target_lang = "indo"
prefix = "translate English to Indonesia: "


def preprocess_function(examples):
    inputs = [prefix + example for example in examples[source_lang]]
    targets = [example for example in examples[target_lang]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [ ]:
tokenized_data = data_load.map(preprocess_function, batched=True)

Map:   0%|          | 0/28989 [00:00<?, ? examples/s]

Map:   0%|          | 0/7248 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

In [ ]:
metric = evaluate.load("sacrebleu")

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_data["train"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_data["test"],
    shuffle=False,
    batch_size=8,
    collate_fn=data_collator,
)

In [ ]:
model.compile(optimizer=optimizer, metrics=["accuracy"])

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

In [ ]:
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_test_set)

In [ ]:
callbacks = [metric_callback]

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=10)

Epoch 1/10
3623/3623 [==============================] - 401s 111ms/step - loss: 1.5041 - accuracy: 0.3861 - val_loss: 1.2415 - val_accuracy: 0.4152
Epoch 2/10
3623/3623 [==============================] - 398s 110ms/step - loss: 1.4434 - accuracy: 0.3930 - val_loss: 1.1915 - val_accuracy: 0.4205
Epoch 3/10
3623/3623 [==============================] - 393s 108ms/step - loss: 1.3859 - accuracy: 0.3981 - val_loss: 1.1493 - val_accuracy: 0.4244
Epoch 4/10
3623/3623 [==============================] - 398s 110ms/step - loss: 1.3352 - accuracy: 0.4032 - val_loss: 1.1020 - val_accuracy: 0.4295
Epoch 5/10
3623/3623 [==============================] - 394s 109ms/step - loss: 1.2875 - accuracy: 0.4079 - val_loss: 1.0695 - val_accuracy: 0.4331
Epoch 6/10
3623/3623 [==============================] - 395s 109ms/step - loss: 1.2425 - accuracy: 0.4121 - val_loss: 1.0366 - val_accuracy: 0.4370
Epoch 7/10
2365/3623 [==================>...........] - ETA: 2:03 - loss: 1.2107 - accuracy: 0.4155

KeyboardInterrupt: 

In [ ]:
model.save_pretrained("translate-eng-indo")

# Model Inference

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
text = "translate English to Indonesia: hungry."

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('t5-small')
inputs = tokenizer(text, return_tensors="tf").input_ids

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/translator-eng-indo")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/drive/MyDrive/translator-eng-indo.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'Lapar.'